In [1]:
# filelist
import platform
import cv2
import numpy as np
import os
from pathlib import Path

if platform.system()=='Darwin':
  root = os.getcwd() # Don't change this
  data_dirname = '/data_test/plantvillage/' # Change as you like 
  p = Path(root + data_dirname)
  p.mkdir(exist_ok=True) 
else:
  p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
  p.mkdir(exist_ok=True)


classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"
  ]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))
# Pick first image of first class (i.e., cat)
# filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
print(len(filelist))

Apple___Cedar_apple_rust Apple___Apple_scab Apple___Black_rot Apple___healthy 1645


In [2]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(40, 40, 3))
type(conv_base)

2022-10-17 10:09:28.858214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2022-10-17 10:09:41.319156: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


keras.engine.functional.Functional

In [58]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda, Layer
from keras.models import Model
from src.fast_soft_sort.tf_ops import soft_rank
import tensorflow as tf
#Custom ranking layer
class Ranking(Layer):
    def __init__(self, regularization_strength = 1):
        self.regularization_strength = regularization_strength
        super(Ranking, self).__init__()

    def call(self, inputs):
        
        return soft_rank(inputs, regularization_strength = self.regularization_strength)

In [72]:
class TestL(Layer):
    def __init__(self, regularization_strength = 1):
        self.regularization_strength = regularization_strength
        super(TestL, self).__init__()

    def call(self, inputs):
        
        return tf.tensordot(inputs,tf.convert_to_tensor([1.2,2,3]),0)

In [73]:
from re import L
import tensorflow as tf

out = Input(3)
print(out)
Layer = TestL(0.1)(out)
print(Layer)
end = Dense(3)(Layer)
print(end)

KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_19'), name='input_19', description="created by layer 'input_19'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3), dtype=tf.float32, name=None), name='test_l_4/Tensordot:0', description="created by layer 'test_l_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3), dtype=tf.float32, name=None), name='dense_23/BiasAdd:0', description="created by layer 'dense_23'")


In [56]:
from re import L
import tensorflow as tf

out = Input(3)
print(out)
Layer = Ranking(0.1)(out)
print(Layer)
end = Dense(3)(Layer)
print(end)

KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_14'), name='input_14', description="created by layer 'input_14'")


AssertionError: Exception encountered when calling layer "ranking_34" (type Ranking).

in user code:

    File "/var/folders/l9/9d49h4t14s52dq6c276zwhgc0000gn/T/ipykernel_25950/4059561856.py", line 13, in call  *
        return tf.numpy_function(soft_rank(inputs, regularization_strength = self.regularization_strength))
    File "/Users/jonaspedersen/Dokumenter/Visual_rec/project/src/fast_soft_sort/tf_ops.py", line 73, in soft_rank  *
        assert tf.executing_eagerly()

    AssertionError: 


Call arguments received by layer "ranking_34" (type Ranking):
  • inputs=tf.Tensor(shape=(None, 3), dtype=float32)

In [74]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda, Layer
from keras.models import Model
from src.fast_soft_sort.tf_ops import soft_rank


number_of_tiles = 9
image_net = conv_base
tiles = Input((9,40,40,3))
tile1 = Lambda(lambda x: x[:,0,:,:,:])(tiles) 
tile2 = Lambda(lambda x: x[:,1,:,:,:])(tiles) 
tile3 = Lambda(lambda x: x[:,2,:,:,:])(tiles) 
tile4 = Lambda(lambda x: x[:,3,:,:,:])(tiles) 
tile5 = Lambda(lambda x: x[:,4,:,:,:])(tiles) 
tile6 = Lambda(lambda x: x[:,5,:,:,:])(tiles) 
tile7 = Lambda(lambda x: x[:,6,:,:,:])(tiles) 
tile8 = Lambda(lambda x: x[:,7,:,:,:])(tiles) 
tile9 = Lambda(lambda x: x[:,8,:,:,:])(tiles) 
tile1 = preprocess_input(tile1) 
tile2 = preprocess_input(tile2) 
tile3 = preprocess_input(tile3) 
tile4 = preprocess_input(tile4) 
tile5 = preprocess_input(tile5) 
tile6 = preprocess_input(tile6) 
tile7 = preprocess_input(tile7) 
tile8 = preprocess_input(tile8) 
tile9 = preprocess_input(tile9) 

embedding_of_tile1 = image_net(tile1) 
embedding_of_tile2 = image_net(tile2)
embedding_of_tile3 = image_net(tile3) 
embedding_of_tile4 = image_net(tile4) 
embedding_of_tile5 = image_net(tile5) 
embedding_of_tile6 = image_net(tile6)
embedding_of_tile7 = image_net(tile7)
embedding_of_tile8 = image_net(tile8)
embedding_of_tile9 = image_net(tile9)

concatonation = Concatenate(axis=3)([
    embedding_of_tile1,
    embedding_of_tile2,
    embedding_of_tile3,
    embedding_of_tile4,
    embedding_of_tile5,
    embedding_of_tile6,
    embedding_of_tile7,
    embedding_of_tile8,
    embedding_of_tile9
    ])

out = Dense(500, activation="relu")(concatonation)
# out = Dense(500, activation="relu")(out)
out = Dense(9)(out)
out = Flatten()(out)
out = Lambda(lambda x: soft_rank(x, regularization_strength=0.5))(out)

model = Model(inputs=tiles, outputs=out)

AssertionError: Exception encountered when calling layer "lambda_28" (type Lambda).



Call arguments received by layer "lambda_28" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 9), dtype=float32)
  • mask=None
  • training=False

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 9, 40, 40,   0           []                               
                                3)]                                                               
                                                                                                  
 lambda (Lambda)                (None, 40, 40, 3)    0           ['input_2[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 40, 40, 3)    0           ['input_2[0][0]']                
                                                                                                  
 lambda_2 (Lambda)              (None, 40, 40, 3)    0           ['input_2[0][0]']            

In [7]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:-3]:
    layer.trainable=False
for layer in model.layers[-3:]:
    layer.trainable=True

Total number of layers is 33
Number of pretrained base layers is 86


In [8]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=tfr.keras.losses.ListMLELoss(), metrics=['binary_accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 15 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=15)

/var/folders/l9/9d49h4t14s52dq6c276zwhgc0000gn/T/ipykernel_25950/1339731381.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/15
19/19 [==============================] - 38s 1s/step - loss: 12.8557 - binary_accuracy: 0.1111 - val_loss: 12.8177 - val_binary_accuracy: 0.1111
Epoch 2/15
19/19 [==============================] - 16s 846ms/step - loss: 12.8164 - binary_accuracy: 0.1111 - val_loss: 12.8166 - val_binary_accuracy: 0.1111
Epoch 3/15
19/19 [==============================] - 17s 911ms/step - loss: 12.8103 - binary_accuracy: 0.1111 - val_loss: 12.8169 - val_binary_accuracy: 0.1111
Epoch 4/15
19/19 [==============================] - 15s 797ms/step - loss: 12.8111 - binary_accuracy: 0.1111 - val_loss: 12.8066 - val_binary_accuracy: 0.1111
Epoch 5/15
19/19 [==============================] - 15s 771ms/step - loss: 12.8097 - binary_accuracy: 0.1111 - val_loss: 12.8105 - val_binary_accuracy: 0.1111
Epoch 6/15
19/19 [==============================] - 15s 804ms/step - loss: 12.8103 - binary_accuracy: 0.1111 - val_loss: 12.8068 - val_binary_accuracy: 0.1111


KeyboardInterrupt: 

In [ ]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train)
validation_generator = PermNetDataGenerator(x_test)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss=tfr.keras.losses.MeanSquaredLoss(), metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 15 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=15)


In [ ]:
newDataGen = PermNetDataGenerator(input=x_train, batch_size=4, preprocess_func=preprocess_input)

In [ ]:
X, y = next(newDataGen)

In [ ]:
showPermImg(X[0],y[0])

In [ ]:
test_gen = PermNetDataGenerator(x_test) # never do this
X, y = next(test_gen)
model.predict(np.expand_dims(X[0], axis=0))